In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/home/diego/local_repos/AGEC652_2024/AGEC652_2024/slides/lecture_6_3"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| include: false
using Pkg
Pkg.activate(".")
Pkg.instantiate()
# Pkg.add("JuMP")
# Pkg.add("Ipopt")
# Pkg.add("Statistics")
# Pkg.add("Plots")
# Pkg.add("ForwardDiff")
# Pkg.add("LinearAlgebra")
# Pkg.add("LaTeXStrings")
# Pkg.add("GLM")
# Pkg.add("DataFrames")
# Pkg.add("Distributions")

  Activating new project at `~/local_repos/AGEC652_2024/AGEC652_2024/slides/lecture_6_3`


In [3]:
β_b = [-2.5, -1.2,  0.2, -0.1,  0.2,  0.0, -0.6]
β_t = [-1.0, -1.2, -0.1,  0.3,  0.4, -0.1, -0.2] 

7-element Vector{Float64}:
 -1.0
 -1.2
 -0.1
  0.3
  0.4
 -0.1
 -0.2

In [4]:
N_survey = 200 # Individuals surveyed per city
N_nr_cities = 30 # Non-rail cities surveyed

30

In [5]:
using Random, Distributions, Statistics
Random.seed!(652)
density_nr = rand(Uniform(1, 8), N_nr_cities); # Thousands of people per square mile

In [6]:
busroutes_nr = density_nr .* 6 .+ rand(Uniform(-3, 3), N_nr_cities) # 6 x density plus random variation
busroutes_nr = round.(busroutes_nr, digits = 0) # Make it an integer

30-element Vector{Float64}:
 31.0
 44.0
 27.0
 39.0
 22.0
 13.0
 12.0
 45.0
 17.0
 34.0
 13.0
 46.0
 19.0
  ⋮
  7.0
 11.0
 40.0
 11.0
 22.0
 11.0
 45.0
 20.0
 25.0
 13.0
 28.0
 15.0

In [7]:
price_nr = busroutes_nr .* 10 .+ rand(Uniform(-10, 10), N_nr_cities) # 10 x routs plus random variation

30-element Vector{Float64}:
 311.8107371553421
 441.836191808295
 264.8818539407205
 397.85047467499123
 218.32637536719224
 120.61369295252364
 123.35849260967757
 443.24246154259276
 177.56090263370731
 340.33962792691113
 131.65971058326127
 462.91066811151245
 185.64982357258796
   ⋮
  75.44694091698362
 110.04883711504074
 401.3601667879812
 109.83944412710363
 212.0424143311709
 118.87771177995403
 443.34874928624066
 191.39724602370902
 259.5257358178362
 134.8663222628205
 270.44782000854934
 159.92212716326662

In [8]:
using DataFrames
data_nr_cities = DataFrame(
  city = ["nr_$(i)" for i in 1:N_nr_cities], # Create an index
  price = price_nr,
  busroutes = busroutes_nr, 
  trainstops = zeros(N_nr_cities), # No trains
  density = density_nr
);
println(data_nr_cities)

30×5 DataFrame
 Row │ city    price     busroutes  trainstops  density 
     │ String  Float64   Float64    Float64     Float64 
─────┼──────────────────────────────────────────────────
   1 │ nr_1    311.811        31.0         0.0  5.30334
   2 │ nr_2    441.836        44.0         0.0  7.70467
   3 │ nr_3    264.882        27.0         0.0  4.35255
   4 │ nr_4    397.85         39.0         0.0  6.29315
   5 │ nr_5    218.326        22.0         0.0  4.07286
   6 │ nr_6    120.614        13.0         0.0  2.42885
   7 │ nr_7    123.358        12.0         0.0  1.87182
   8 │ nr_8    443.242        45.0         0.0  7.32123
   9 │ nr_9    177.561        17.0         0.0  2.33616
  10 │ nr_10   340.34         34.0         0.0  5.5303
  11 │ nr_11   131.66         13.0         0.0  2.06674
  12 │ nr_12   462.911        46.0         0.0  7.38693
  13 │ nr_13   185.65         19.0         0.0  2.84135
  14 │ nr_14   388.729        39.0         0.0  6.95232
  15 │ nr_15   462.734        4